## PDI - Rebalancing data for November 2022

## Setting

In [793]:
import pandas as pd
import numpy as np
import requests
import os
import decouple

In [794]:
import time

In [795]:
from pycoingecko import CoinGeckoAPI
key =  decouple.config("CG_KEY")
cg = CoinGeckoAPI(api_key=key)

In [796]:
from defillama import DefiLlama
llama = DefiLlama()

In [797]:
import plotly.express as px
import plotly.graph_objects as go

In [798]:
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)

## Data retrieving and filtering

### Token inclusion criteria

1. **Project and token characteristics** 
    1. The project must have a listing on Defi Llama
    1. The project’s token should have been listed on CoinGecko with pricing data at least 1 year prior to the date of inclusion in the index. 
    1. The project should have a token that is native to Ethereum L1 or L2. This excludes wrapped variants, where the underlying tokens are locked on an alt-L1.
    1. The project should be a going concern, with a dedicated team actively building, supporting and maintaining the project. 
    1. No rebasing or deflationary tokens.
    1. The project must be widely considered to be building a useful protocol or product. Projects that have ponzi characteristics at the core of their offering will not be considered.
    1. Synthetic tokens which derive their value from external price feeds are not permissible. 
    1. The project’s token must not have the ability to pause token transfers.
    1. The project’s protocol or product must have significant usage.
<br><br>
1. **Pricing requirmeents**
    1. Token should have a manipulation resistant price feed in the form of a Chainlink price feed or a Uniswap V3 TWAP oracle with a cardinality of at least 300 and a minimum liquidity level of $1mm.
<br><br>
1. **Token Supply requirements**
    1. The project's token must have a circulating supply greater than 30% of the max supply. In cases where a token does not have a max supply, the minting mechanics would need to be assessed.
<br><br>
1. **Liquidity Requirements**
    1. The token must be listed on a supported exchange.
    1. The token should have in aggregate at least \$5mm of onchain liquidity across Uniswap v2, Uniswap v3, Sushiswap, Balancer v1, Balancer v2, Bancor v2 and Bancor v3.
    1. The price should experience no more than 4% price impact when executing a $100k trade.
    1. The token must have shown consistent DeFi liquidity on Ethereum.
<br><br>
1. **Security Requirements**
    1. The project must have been audited by smart contract security professionals with the audit report(s) publicly available. Alternatively, the protocol must have been operating long enough to create a consensus about its safety in the decentralised finance community.

### Tokens list

In [799]:
def normalize_symb(symb):
    return ''.join(c for c in symb if c.isalnum()).upper()

#### DefiLlama

In [800]:
protocols = llama.get_all_protocols()

In [801]:
top_defi_tokens = {} 
for pr in protocols[:1000]:
    if 'Ethereum' in pr['chains'] and pr['address'] and pr['address']!='-' and pr['symbol'] != '-':
        symbol = normalize_symb(pr['symbol']) 
        address = pr['address'].lower().split(":")[-1]
        top_defi_tokens[symbol] = {
            'symbol': symbol, 
            'address': address 
        }

#### CoinGecko

##### Tokens addresses

In [802]:
coins_list = pd.DataFrame(cg.get_coins_list(include_platform=True))

In [803]:
tokens = {} 
tokens_by_addr = {}
for index, coin in coins_list.iterrows():
    if len(coin['platforms']) >= 1 and 'ethereum' in list(coin['platforms'].keys()):
        symbol = normalize_symb(coin['symbol'])
        address = coin['platforms']['ethereum'].lower().split(":")[-1]
        token = {
            'symbol': symbol,
            'address': address 
        }
        tokens[symbol] = token
        tokens_by_addr[address] = token

In [804]:
tokens['USDC'], tokens['WETH']

({'symbol': 'USDC', 'address': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48'},
 {'symbol': 'WETH', 'address': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2'})

In [805]:
# These token's addresses cant be found in the coingecko list of eth addresses
exclude_symbols = []
for symb, token in top_defi_tokens.items():
    if token['address'] not in tokens_by_addr.keys():
        # print(token)
        exclude_symbols.append(token['symbol'])

In [806]:
for symb in exclude_symbols:
    del top_defi_tokens[symb]

##### Market data

In [807]:
cg_tokens_data = cg.get_coins_markets(vs_currency='USD', order='market_cap_desc')
for i in range(2, 20):
    cg_tokens_data.extend(cg.get_coins_markets(vs_currency='USD', order='market_cap_desc', page=i))
    
cg_tokens_data = pd.DataFrame.from_records(cg_tokens_data)

In [808]:
tokens_market_data = cg_tokens_data.set_index(cg_tokens_data['symbol'].str.upper())

In [809]:
duplicated = tokens_market_data.index[tokens_market_data.index.duplicated(keep='first')]
tokens_market_data = tokens_market_data.drop(index=duplicated)

In [810]:
tokens_market_data = tokens_market_data.rename(index={"ETH": "WETH"})

### Token's supply check 

In [811]:
sp_passed_index = (tokens_market_data['circulating_supply'] / tokens_market_data['total_supply']) > 0.3
sp_passed = tokens_market_data[sp_passed_index]

### Liquidity check

In [812]:
api_urls = {
    'univ2': "https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2",
    'univ3': "https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3",
    'sushi': "https://api.thegraph.com/subgraphs/name/sushiswap/exchange",
    'bancorv2': "https://api-v2.bancor.network/tokens",
    'bancorv3': "https://api-v3.bancor.network/pools",
    'balancer': "https://api.thegraph.com/subgraphs/name/balancer-labs/balancer",
    'balancerv2': "https://api.thegraph.com/subgraphs/name/balancer-labs/balancer-v2" 
} 

In [813]:
def univ2_get_liquidities(tokens_ids):
    query = """
        query pairs($tokens_ids: [ID!]!) {
            pairs (first:1000, 
                where: {
                    token0_in: $tokens_ids, 
                    token1_in: $tokens_ids,
                    reserveUSD_gt: 10000
                }){
                token0 {
                    id
                    symbol
                }
                token1 {
                    id
                    symbol
                }
                reserveUSD
            }
        }
    """
    url = api_urls['univ2']
    
    response = requests.post(url, json={'query': query, 'variables': {'tokens_ids': tokens_ids}})
    pairs = response.json()['data']['pairs']
    
    liqs = {}
    for pair in pairs:
        liq = float(pair['reserveUSD'])
        
        symb0 = normalize_symb(pair['token0']['symbol'])
        if symb0 in tokens_market_data.index and pair['token0']['id'].lower() in tokens_ids:
            liqs[symb0] = liqs.get(symb0, 0) + liq
        
        symb1 = normalize_symb(pair['token1']['symbol'])
        if symb1 in tokens_market_data.index and pair['token1']['id'].lower() in tokens_ids:
            liqs[symb1] = liqs.get(symb1, 0) + liq 
        
    return liqs
    

In [814]:
def univ3_get_liquidities(tokens_ids):
    query = """
        query pools($tokens_ids: [ID!]!) {
            pools (first:1000, 
                where: {
                    token0_in: $tokens_ids, 
                    token1_in: $tokens_ids,
                    totalValueLockedUSD_gt: 10000
                }){
                token0 {
                    id
                    symbol
                } 
                token1 {
                    id
                    symbol
                }
                totalValueLockedUSD
            }
        }
    """
    url = api_urls['univ3']
    
    response = requests.post(url, json={'query': query, 'variables': {'tokens_ids': tokens_ids}})
    pools = response.json()['data']['pools']
        
    liqs = {}
    for pool in pools:
        liq = float(pool['totalValueLockedUSD'])
        
        symb0 = normalize_symb(pool['token0']['symbol'])
        if symb0 in tokens_market_data.index and pool['token0']['id'].lower() in tokens_ids:
            liqs[symb0] = liqs.get(symb0, 0) + liq 
        
        symb1 = normalize_symb(pool['token1']['symbol'])
        if symb1 in tokens_market_data.index and pool['token1']['id'].lower() in tokens_ids:
            liqs[symb1] = liqs.get(symb1, 0) + liq 
            
    return liqs



In [815]:
def sushi_get_liquidities(tokens_ids):
    query = """
        query pairs($tokens_ids: [ID!]!) {
            pairs (first:1000, 
                where: {
                    token0_in: $tokens_ids, 
                    token1_in: $tokens_ids,
                    reserveUSD_gt: 10000
                }){
                token0 {
                    id
                    symbol
                }
                token1 {
                    id
                    symbol
                }
                reserveUSD
            }
        }
    """
    url = api_urls['sushi']
    
    response = requests.post(url, json={'query': query, 'variables': {'tokens_ids': tokens_ids}})
    pairs = response.json()['data']['pairs']
    
    liqs = {}
    for pair in pairs:
        liq = float(pair['reserveUSD'])
        
        symb0 = normalize_symb(pair['token0']['symbol'])
        if symb0 in tokens_market_data.index and pair['token0']['id'].lower() in tokens_ids:
            liqs[symb0] = liqs.get(symb0, 0) + liq
        
        symb1 = normalize_symb(pair['token1']['symbol'])
        if symb1 in tokens_market_data.index and pair['token1']['id'].lower() in tokens_ids:
            liqs[symb1] = liqs.get(symb1, 0) + liq
        
    return liqs
    

In [816]:
def bancorv2_get_liquidities(tokens_ids):
    url = api_urls['bancorv2'] 
    response = requests.get(url)
    tokens = response.json()['data']
    
    liqs = {}
    for token in tokens:
        if token['dlt_id'].lower() in tokens_ids:
            symb = normalize_symb(token['symbol'])
            liqs[symb] = float(token['liquidity']['usd'])
            
    return liqs
            

In [817]:
def bancorv3_get_liquidities(tokens_ids):
    url = api_urls['bancorv3'] 
    response = requests.get(url)
    pools = response.json()['data']
    
    liqs = {}
    for pool in pools:
        if pool['poolDltId'].lower() in tokens_ids:
            symb = normalize_symb(pool['name'])
            liqs[symb] = liqs.get(symb, 0) + float(pool['liquidity']['usd'])
            
    return liqs
            

In [818]:
def balancer_get_liquidities(tokens_ids):
    query = """
        query pools {
            pools (
                first: 1000,
                where: {
                  liquidity_gt: 10000
                }
            ){
                tokens {
                  address
                  symbol
                  balance
                }
                liquidity
          }
        }
    """
    url = api_urls['balancer']
    
    response = requests.post(url, json={'query': query})
    pools = response.json()['data']['pools']
    
    liqs = {}
    for pool in pools:
        liq = 0
        for token in pool['tokens']:
            symb = normalize_symb(token['symbol'])
            if symb in tokens_market_data.index: 
                liq += float(token['balance']) * tokens_market_data.loc[symb]['current_price']
            
        for token in pool['tokens']:
            symb = normalize_symb(token['symbol'])
            if symb not in tokens_market_data.index or token['address'].lower() not in tokens_ids:
                continue
                
            liqs[symb] = liqs.get(symb, 0) + liq
        
    return liqs
    

In [819]:
def balancerv2_get_liquidities(tokens_ids):
    query = """
        query pools {
            pools (
                first: 1000,
                where: {
                  totalLiquidity_gt: 10000
                }
            ){
                tokens {
                  address
                  symbol
                  balance
                }
                totalLiquidity
          }
        }
    """
    url = api_urls['balancerv2']
    
    response = requests.post(url, json={'query': query})
    pools = response.json()['data']['pools']
    
    liqs = {}
    for pool in pools:
        liq = float(pool['totalLiquidity'])
        
        for token in pool['tokens']:
            symb = normalize_symb(token['symbol'])
            if symb not in tokens_market_data.index or token['address'].lower() not in tokens_ids:
                continue
            
            liqs[symb] = liqs.get(symb, 0) + liq 
        
    return liqs
    

In [820]:
tokens['AMP']

{'symbol': 'AMP', 'address': '0xff20817765cb7f73d4bde2e66e067e58d11095c2'}

In [821]:
tokens_ids = [token['address'] for symb, token in top_defi_tokens.items()]

In [822]:
tokens_ids.append(tokens['WETH']['address'])
tokens_ids.append(tokens['USDC']['address'])

In [823]:
query_functions = {
    'univ2': univ2_get_liquidities,
    'univ3': univ3_get_liquidities, 
    'sushi': sushi_get_liquidities, 
    'bancorv2': bancorv2_get_liquidities,
    'bancorv3': bancorv3_get_liquidities,
    'balancer': balancer_get_liquidities, 
    'balancerv2': balancerv2_get_liquidities, 
}

In [824]:
liqs_dict = {}
for dex, func in query_functions.items():
    liqs_dict[dex] = func(tokens_ids)

In [825]:
liquidities = pd.DataFrame.from_dict(liqs_dict)

#Adjusting RBN liquidity due to subgraph error. RBN/WETH pool is overestimated by 10x on the subgraph
univ3_url = api_urls["univ3"]
query = """query{
pool(id:"0x94981f69f7483af3ae218cbfe65233cc3c60d93a"){
  totalValueLockedUSD
}
}"""
response = requests.post(univ3_url, json={'query': query})
tvl = float(response.json()['data']['pool']["totalValueLockedUSD"])
liquidities.loc[["RBN"],["univ3"]]= liquidities.loc[["RBN"],["univ3"]]["univ3"] - tvl +tvl/10


In [826]:
liquidities['sum'] = liquidities.sum(axis=1)
liquidities.sort_values('sum', ascending=False).head(50)
liquidities

,univ2,univ3,sushi,bancorv2,bancorv3,balancer,balancerv2,sum
WBTC,1.088755e+07,4.532942e+08,2.049463e+07,3.522167e+06,3.284346e+06,4.112967e+06,2.081514e+07,5.164110e+08
USDC,1.065775e+08,8.934663e+08,6.351904e+07,5.676262e+05,1.313723e+06,1.790018e+07,8.174422e+07,1.165089e+09
DOP,4.501446e+05,NaN,NaN,NaN,NaN,NaN,NaN,4.501446e+05
WETH,1.838066e+08,1.189781e+09,1.772653e+08,NaN,NaN,1.181539e+08,7.139053e+08,2.382913e+09
AMP,8.422235e+04,5.197773e+05,1.396975e+05,2.494891e+04,1.831546e+04,NaN,NaN,7.869616e+05
...,...,...,...,...,...,...,...,...
MIMO,NaN,NaN,NaN,NaN,NaN,2.652534e+04,7.381750e+04,1.003428e+05
TEMPLE,NaN,NaN,NaN,NaN,NaN,NaN,9.520004e+06,9.520004e+06
NOTE,NaN,NaN,NaN,NaN,NaN,NaN,4.478764e+06,4.478764e+06
SOLACE,NaN,NaN,NaN,NaN,NaN,NaN,2.575554e+05,2.575554e+05


In [827]:
liq_check = (liquidities['sum'] > 5e6).sort_values(ascending=False)
liq_check_passed = liq_check[liq_check]

In [828]:
lq_passed = sp_passed[
    sp_passed.index.isin(liq_check_passed.index)
]

### Non quantative checks

In [829]:
exclude_list = [
    {
        'symbol': 'WBTC',
        'reason': 'Criteria 1.C'
    },
    {
        'symbol': 'WETH',
        'reason': 'Criteria 1.C'
    },
    {
        'symbol': 'ERN',
        'reason': 'Not DeFi'
    },
    {
        'symbol': 'NFTX',
        'reason': 'Not DeFi'
    },
    {
        'symbol': 'FXS',
        'reason': 'Criteria 1.G'
    },
    {
        'symbol': 'RGT',
        'reason': 'Deprecated asset'
    },
    {
        'symbol': 'SUPER',
        'reason': 'Not DeFi'
    },
    {
        'symbol': 'BOND',
        'reason': 'Criteria 1.I'
    },
    {
        'symbol': 'USDC',
        'reason': 'stable'
    },
    {
        'symbol': 'FEI',
        'reason': 'stable'
    },
    {
        'symbol': 'SYN',
        'reason': 'bridge'
    },
    
]
exclude_symbols = [token['symbol'] for token in exclude_list]

In [830]:
nq_passed = lq_passed[
    ~lq_passed.index.isin(exclude_symbols)
]

In [831]:
nq_passed

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
symbol,,,,,,,,,,,,,,,,,,,,,
UNI,uniswap,uni,Uniswap,https://assets.coingecko.com/coins/images/1250...,7.110000,5357665333,18,7.107857e+09,194867920.0,7.110000,...,1.000000e+09,1.000000e+09,44.92,-84.24234,2021-05-03T05:25:04.822Z,1.030000,587.07402,2020-09-17T01:20:38.214Z,None,2022-10-29T17:12:12.506Z
AAVE,aave,aave,Aave,https://assets.coingecko.com/coins/images/1264...,86.310000,1220004205,46,1.380995e+09,107351690.0,86.400000,...,1.600000e+07,1.600000e+07,661.69,-86.97151,2021-05-18T21:19:59.514Z,26.020000,231.27523,2020-11-05T09:20:11.928Z,None,2022-10-29T17:12:00.368Z
LDO,lido-dao,ldo,Lido DAO,https://assets.coingecko.com/coins/images/1357...,1.670000,1196956585,50,1.670405e+09,30100103.0,1.750000,...,1.000000e+09,1.000000e+09,7.30,-77.14994,2021-08-20T08:35:20.158Z,0.406150,310.86092,2022-06-18T20:55:12.035Z,None,2022-10-29T17:12:01.042Z
MKR,maker,mkr,Maker,https://assets.coingecko.com/coins/images/1364...,918.410000,827750775,62,9.235072e+08,32391728.0,927.480000,...,9.776310e+05,1.005577e+06,6292.31,-85.42922,2021-05-03T21:54:29.333Z,168.360000,444.57842,2020-03-16T20:52:36.527Z,None,2022-10-29T17:11:59.939Z
SNX,havven,snx,Synthetix Network,https://assets.coingecko.com/coins/images/3406...,2.560000,615320068,77,7.791542e+08,59402707.0,2.560000,...,3.045289e+08,3.045289e+08,28.53,-91.01548,2021-02-14T01:12:38.505Z,0.034786,7269.22587,2019-01-06T00:00:00.000Z,"{'times': 4.110756055391768, 'currency': 'usd'...",2022-10-29T17:12:03.597Z
CRV,curve-dao-token,crv,Curve DAO,https://assets.coingecko.com/coins/images/1212...,0.958764,537494338,84,3.167342e+09,56588597.0,0.971458,...,1.844116e+09,3.303030e+09,15.37,-93.74854,2020-08-14T00:00:00.000Z,0.331577,189.82119,2020-11-05T13:09:50.181Z,None,2022-10-29T17:11:58.770Z
RPL,rocket-pool,rpl,Rocket Pool,https://assets.coingecko.com/coins/images/2090...,24.580000,464174760,96,4.641748e+08,2154556.0,25.090000,...,1.890000e+07,1.890000e+07,59.46,-58.72916,2021-11-09T04:03:31.003Z,0.008847,277253.03296,2018-08-28T00:00:00.000Z,None,2022-10-29T17:12:11.072Z
CVX,convex-finance,cvx,Convex Finance,https://assets.coingecko.com/coins/images/1558...,5.790000,404904833,108,5.787206e+08,7195464.0,5.860000,...,9.553807e+07,1.000000e+08,60.09,-90.40603,2022-01-01T18:04:03.030Z,1.900000,202.95133,2021-07-20T13:17:29.183Z,None,2022-10-29T17:12:08.713Z
1INCH,1inch,1inch,1inch,https://assets.coingecko.com/coins/images/1346...,0.628980,391025599,109,9.438744e+08,30906323.0,0.630424,...,1.500000e+09,1.500000e+09,8.65,-92.73420,2021-10-27T08:24:54.808Z,0.519281,21.05384,2022-06-18T20:10:15.819Z,None,2022-10-29T17:11:59.272Z


### Retrieve historical data & livetime check

In [832]:
exclude_list = []

prices_data = pd.DataFrame()
marketcaps = pd.DataFrame() 
for index, data in nq_passed[['id', 'symbol']].iterrows():
    id_ = data['id']
    symbol = data['symbol'].upper()
    data = cg.get_coin_market_chart_by_id(id_, vs_currency='USD', days='max')
  
    df_prices = pd.DataFrame(data['prices'], columns=['date', symbol])
    df_prices = df_prices[df_prices[symbol] > 0]
    df_prices['date'] = pd.to_datetime(df_prices['date'], unit='ms').dt.date
    df_prices['date'] = pd.to_datetime(df_prices['date'])
    df_prices = df_prices.set_index('date', drop=True)
    df_prices = df_prices.loc[~df_prices.index.duplicated(keep='first')]
    
    if len(df_prices) < 365:
        print(f'Excluding {symbol}, prices data available only for {len(df_prices)} < 365 days')
        exclude_list.append(symbol)
        continue
    prices_data = pd.concat([prices_data, df_prices], axis=1)
    
    df_mcaps = pd.DataFrame(data['market_caps'], columns=['date', symbol])
    df_mcaps = df_mcaps[df_mcaps[symbol] > 0]
    df_mcaps['date'] = pd.to_datetime(df_mcaps['date'], unit='ms').dt.date
    df_mcaps['date'] = pd.to_datetime(df_mcaps['date'])
    df_mcaps = df_mcaps.set_index('date', drop=True)
    df_mcaps = df_mcaps.loc[~df_mcaps.index.duplicated(keep='first')]
    
    if len(df_mcaps) < 365:
        print(f'Note: {symbol}, marketcap data available only for {len(df_mcaps)} < 365 days')
        ## continue
    df_mcaps = df_mcaps.reindex(df_prices.index)
    marketcaps = pd.concat([marketcaps, df_mcaps], axis=1)
    
    

Excluding TEMPLE, prices data available only for 291 < 365 days


In [833]:
lt_passed = nq_passed[
    ~nq_passed.index.isin(exclude_list)
]

In [834]:
lt_passed

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
symbol,,,,,,,,,,,,,,,,,,,,,
UNI,uniswap,uni,Uniswap,https://assets.coingecko.com/coins/images/1250...,7.110000,5357665333,18,7.107857e+09,194867920.0,7.110000,...,1.000000e+09,1.000000e+09,44.92,-84.24234,2021-05-03T05:25:04.822Z,1.030000,587.07402,2020-09-17T01:20:38.214Z,None,2022-10-29T17:12:12.506Z
AAVE,aave,aave,Aave,https://assets.coingecko.com/coins/images/1264...,86.310000,1220004205,46,1.380995e+09,107351690.0,86.400000,...,1.600000e+07,1.600000e+07,661.69,-86.97151,2021-05-18T21:19:59.514Z,26.020000,231.27523,2020-11-05T09:20:11.928Z,None,2022-10-29T17:12:00.368Z
LDO,lido-dao,ldo,Lido DAO,https://assets.coingecko.com/coins/images/1357...,1.670000,1196956585,50,1.670405e+09,30100103.0,1.750000,...,1.000000e+09,1.000000e+09,7.30,-77.14994,2021-08-20T08:35:20.158Z,0.406150,310.86092,2022-06-18T20:55:12.035Z,None,2022-10-29T17:12:01.042Z
MKR,maker,mkr,Maker,https://assets.coingecko.com/coins/images/1364...,918.410000,827750775,62,9.235072e+08,32391728.0,927.480000,...,9.776310e+05,1.005577e+06,6292.31,-85.42922,2021-05-03T21:54:29.333Z,168.360000,444.57842,2020-03-16T20:52:36.527Z,None,2022-10-29T17:11:59.939Z
SNX,havven,snx,Synthetix Network,https://assets.coingecko.com/coins/images/3406...,2.560000,615320068,77,7.791542e+08,59402707.0,2.560000,...,3.045289e+08,3.045289e+08,28.53,-91.01548,2021-02-14T01:12:38.505Z,0.034786,7269.22587,2019-01-06T00:00:00.000Z,"{'times': 4.110756055391768, 'currency': 'usd'...",2022-10-29T17:12:03.597Z
CRV,curve-dao-token,crv,Curve DAO,https://assets.coingecko.com/coins/images/1212...,0.958764,537494338,84,3.167342e+09,56588597.0,0.971458,...,1.844116e+09,3.303030e+09,15.37,-93.74854,2020-08-14T00:00:00.000Z,0.331577,189.82119,2020-11-05T13:09:50.181Z,None,2022-10-29T17:11:58.770Z
RPL,rocket-pool,rpl,Rocket Pool,https://assets.coingecko.com/coins/images/2090...,24.580000,464174760,96,4.641748e+08,2154556.0,25.090000,...,1.890000e+07,1.890000e+07,59.46,-58.72916,2021-11-09T04:03:31.003Z,0.008847,277253.03296,2018-08-28T00:00:00.000Z,None,2022-10-29T17:12:11.072Z
CVX,convex-finance,cvx,Convex Finance,https://assets.coingecko.com/coins/images/1558...,5.790000,404904833,108,5.787206e+08,7195464.0,5.860000,...,9.553807e+07,1.000000e+08,60.09,-90.40603,2022-01-01T18:04:03.030Z,1.900000,202.95133,2021-07-20T13:17:29.183Z,None,2022-10-29T17:12:08.713Z
1INCH,1inch,1inch,1inch,https://assets.coingecko.com/coins/images/1346...,0.628980,391025599,109,9.438744e+08,30906323.0,0.630424,...,1.500000e+09,1.500000e+09,8.65,-92.73420,2021-10-27T08:24:54.808Z,0.519281,21.05384,2022-06-18T20:10:15.819Z,None,2022-10-29T17:11:59.272Z


In [835]:
liquidities.loc[lt_passed.index]

,univ2,univ3,sushi,bancorv2,bancorv3,balancer,balancerv2,sum
symbol,,,,,,,,
UNI,2.149407e+07,2.526477e+07,2.788419e+05,8.424265e+04,6.028540e+04,1.144839e+05,1.971580e+06,4.926827e+07
AAVE,1.973782e+05,3.079271e+06,2.542156e+06,5.517452e+04,1.155396e+05,1.091536e+08,5.060784e+04,1.151937e+08
LDO,1.834659e+05,1.357801e+07,7.135843e+06,NaN,NaN,NaN,3.729748e+06,2.462707e+07
MKR,1.869787e+06,7.195556e+06,4.011287e+05,2.529037e+05,1.159358e+05,4.778374e+05,NaN,1.031315e+07
SNX,1.236992e+06,3.861467e+06,7.374614e+05,8.372286e+05,8.988710e+04,1.942584e+05,1.504705e+06,8.461999e+06
CRV,5.223848e+05,7.444140e+06,1.860842e+06,1.899543e+02,0.000000e+00,1.371190e+04,3.528780e+05,1.019415e+07
RPL,NaN,6.612384e+06,NaN,2.749496e+04,6.918475e+04,NaN,1.028585e+07,1.699491e+07
CVX,NaN,4.981527e+06,9.334913e+05,NaN,NaN,NaN,NaN,5.915018e+06
1INCH,2.397319e+05,4.814089e+07,NaN,1.861952e+03,0.000000e+00,NaN,NaN,4.838249e+07


### Slippage check

To derive a slippage threshold, consider an abstract univ2-like pool with $5m\$$ overall liquidity. 
I.e if pool has reserves $R_1$, $R_2$ and $p_1$ and $p_2$ are the correspondent tokens prices (in USD), then
$$R_1p_1 = R_2p_2 = 2.5m \$$$

Swapping $x$ token1 for token2 will result in the output amount $y$ such that
$$R_1\cdot R_2 = (R_1 + x) \cdot (R_2 - y)$$
$$y = \frac{R_2\cdot x}{R_1 + x}$$
Then the delivery price will be
$$p_d = \frac{y}{x} = \frac{R_2}{R_1 + x}$$
The spot price may be estimated as a delivery price for relatively small input amount $x_0$
$$p_s = \frac{y_0}{x_0} = \frac{R_2}{R_1 + x_0}$$

Then the theoretical slippage for this pool will be
$$slippage = \frac{p_d}{p_s} - 1 = \frac{R_1+x_0}{R_1+x} - 1 = \frac{p_1R_1+p_1x_0}{p_1R_1+p_1x}-1$$

In [836]:
liq = 5*10**6
liq_os = liq / 2 # one-side liquidity 
trade_value_tiny = 1e3
trade_value = 1e5

In [837]:
max_slippage = (liq_os + trade_value_tiny) / (liq_os + trade_value) - 1
# round to the nearest .5
max_slippage = round(2 * max_slippage, 2) / 2 
max_slippage

-0.04

In [838]:
url_0x = "https://api.0x.org/swap/v1/quote" 

In [839]:
decimals = 18
slippages = []
for symb, data in lt_passed.iterrows():
    query = {
        'buyToken': 'WETH',
        'sellToken': tokens[symb]['address'],
        'sellAmount': int(trade_value_tiny / data['current_price'] * 10 ** decimals),
    }
    
    # spot price is calculated as a price for 1000$ swap
    resp = requests.get(url_0x, params=query)
    swap = resp.json()
    spot_price = float(swap['price'])
    
    query['sellAmount'] = int(trade_value / data['current_price'] * 10 ** decimals)
    resp = requests.get(url_0x, params=query)
    swap = resp.json()
    del_price = float(swap['price'])
    
    slippage = del_price / spot_price - 1
    
    slippages.append({
        'symb': symb,
        'spot_price': spot_price,
        'delivery_price': del_price,
        'slippage': slippage
    })
    
    

In [840]:
slippages_df = pd.DataFrame(slippages)
slippages_df

,symb,spot_price,delivery_price,slippage
0,UNI,4.330810e-03,4.322642e-03,-0.001886
1,AAVE,5.256230e-02,5.256982e-02,0.000143
2,LDO,1.019106e-03,1.008549e-03,-0.010359
3,MKR,5.599602e-01,5.530327e-01,-0.012371
4,SNX,1.553885e-03,1.522151e-03,-0.020422
5,CRV,5.857582e-04,5.817842e-04,-0.006784
6,RPL,1.496581e-02,1.479952e-02,-0.011112
7,CVX,3.515850e-03,3.513742e-03,-0.000600
8,1INCH,3.843050e-04,3.809873e-04,-0.008633
9,COMP,3.171399e-02,3.092369e-02,-0.024920


In [841]:
exclude_list = list(slippages_df[abs(slippages_df['slippage']) > abs(max_slippage)]['symb'])
exclude_list

['ICHI']

In [842]:
st_passed = lt_passed[
    ~lt_passed.index.isin(exclude_list)
]
st_passed

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
symbol,,,,,,,,,,,,,,,,,,,,,
UNI,uniswap,uni,Uniswap,https://assets.coingecko.com/coins/images/1250...,7.110000,5357665333,18,7.107857e+09,194867920.0,7.110000,...,1.000000e+09,1.000000e+09,44.92,-84.24234,2021-05-03T05:25:04.822Z,1.030000,587.07402,2020-09-17T01:20:38.214Z,None,2022-10-29T17:12:12.506Z
AAVE,aave,aave,Aave,https://assets.coingecko.com/coins/images/1264...,86.310000,1220004205,46,1.380995e+09,107351690.0,86.400000,...,1.600000e+07,1.600000e+07,661.69,-86.97151,2021-05-18T21:19:59.514Z,26.020000,231.27523,2020-11-05T09:20:11.928Z,None,2022-10-29T17:12:00.368Z
LDO,lido-dao,ldo,Lido DAO,https://assets.coingecko.com/coins/images/1357...,1.670000,1196956585,50,1.670405e+09,30100103.0,1.750000,...,1.000000e+09,1.000000e+09,7.30,-77.14994,2021-08-20T08:35:20.158Z,0.406150,310.86092,2022-06-18T20:55:12.035Z,None,2022-10-29T17:12:01.042Z
MKR,maker,mkr,Maker,https://assets.coingecko.com/coins/images/1364...,918.410000,827750775,62,9.235072e+08,32391728.0,927.480000,...,9.776310e+05,1.005577e+06,6292.31,-85.42922,2021-05-03T21:54:29.333Z,168.360000,444.57842,2020-03-16T20:52:36.527Z,None,2022-10-29T17:11:59.939Z
SNX,havven,snx,Synthetix Network,https://assets.coingecko.com/coins/images/3406...,2.560000,615320068,77,7.791542e+08,59402707.0,2.560000,...,3.045289e+08,3.045289e+08,28.53,-91.01548,2021-02-14T01:12:38.505Z,0.034786,7269.22587,2019-01-06T00:00:00.000Z,"{'times': 4.110756055391768, 'currency': 'usd'...",2022-10-29T17:12:03.597Z
CRV,curve-dao-token,crv,Curve DAO,https://assets.coingecko.com/coins/images/1212...,0.958764,537494338,84,3.167342e+09,56588597.0,0.971458,...,1.844116e+09,3.303030e+09,15.37,-93.74854,2020-08-14T00:00:00.000Z,0.331577,189.82119,2020-11-05T13:09:50.181Z,None,2022-10-29T17:11:58.770Z
RPL,rocket-pool,rpl,Rocket Pool,https://assets.coingecko.com/coins/images/2090...,24.580000,464174760,96,4.641748e+08,2154556.0,25.090000,...,1.890000e+07,1.890000e+07,59.46,-58.72916,2021-11-09T04:03:31.003Z,0.008847,277253.03296,2018-08-28T00:00:00.000Z,None,2022-10-29T17:12:11.072Z
CVX,convex-finance,cvx,Convex Finance,https://assets.coingecko.com/coins/images/1558...,5.790000,404904833,108,5.787206e+08,7195464.0,5.860000,...,9.553807e+07,1.000000e+08,60.09,-90.40603,2022-01-01T18:04:03.030Z,1.900000,202.95133,2021-07-20T13:17:29.183Z,None,2022-10-29T17:12:08.713Z
1INCH,1inch,1inch,1inch,https://assets.coingecko.com/coins/images/1346...,0.628980,391025599,109,9.438744e+08,30906323.0,0.630424,...,1.500000e+09,1.500000e+09,8.65,-92.73420,2021-10-27T08:24:54.808Z,0.519281,21.05384,2022-06-18T20:10:15.819Z,None,2022-10-29T17:11:59.272Z


### Marketcap ranking & top10 filtering

In [843]:
chosen_tokens = st_passed.sort_values("market_cap", ascending=False).iloc[:13]

In [844]:
chosen_tokens

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
symbol,,,,,,,,,,,,,,,,,,,,,
UNI,uniswap,uni,Uniswap,https://assets.coingecko.com/coins/images/1250...,7.110000,5357665333,18,7.107857e+09,194867920.0,7.110000,...,1.000000e+09,1.000000e+09,44.92,-84.24234,2021-05-03T05:25:04.822Z,1.030000,587.07402,2020-09-17T01:20:38.214Z,None,2022-10-29T17:12:12.506Z
AAVE,aave,aave,Aave,https://assets.coingecko.com/coins/images/1264...,86.310000,1220004205,46,1.380995e+09,107351690.0,86.400000,...,1.600000e+07,1.600000e+07,661.69,-86.97151,2021-05-18T21:19:59.514Z,26.020000,231.27523,2020-11-05T09:20:11.928Z,None,2022-10-29T17:12:00.368Z
LDO,lido-dao,ldo,Lido DAO,https://assets.coingecko.com/coins/images/1357...,1.670000,1196956585,50,1.670405e+09,30100103.0,1.750000,...,1.000000e+09,1.000000e+09,7.30,-77.14994,2021-08-20T08:35:20.158Z,0.406150,310.86092,2022-06-18T20:55:12.035Z,None,2022-10-29T17:12:01.042Z
MKR,maker,mkr,Maker,https://assets.coingecko.com/coins/images/1364...,918.410000,827750775,62,9.235072e+08,32391728.0,927.480000,...,9.776310e+05,1.005577e+06,6292.31,-85.42922,2021-05-03T21:54:29.333Z,168.360000,444.57842,2020-03-16T20:52:36.527Z,None,2022-10-29T17:11:59.939Z
SNX,havven,snx,Synthetix Network,https://assets.coingecko.com/coins/images/3406...,2.560000,615320068,77,7.791542e+08,59402707.0,2.560000,...,3.045289e+08,3.045289e+08,28.53,-91.01548,2021-02-14T01:12:38.505Z,0.034786,7269.22587,2019-01-06T00:00:00.000Z,"{'times': 4.110756055391768, 'currency': 'usd'...",2022-10-29T17:12:03.597Z
CRV,curve-dao-token,crv,Curve DAO,https://assets.coingecko.com/coins/images/1212...,0.958764,537494338,84,3.167342e+09,56588597.0,0.971458,...,1.844116e+09,3.303030e+09,15.37,-93.74854,2020-08-14T00:00:00.000Z,0.331577,189.82119,2020-11-05T13:09:50.181Z,None,2022-10-29T17:11:58.770Z
RPL,rocket-pool,rpl,Rocket Pool,https://assets.coingecko.com/coins/images/2090...,24.580000,464174760,96,4.641748e+08,2154556.0,25.090000,...,1.890000e+07,1.890000e+07,59.46,-58.72916,2021-11-09T04:03:31.003Z,0.008847,277253.03296,2018-08-28T00:00:00.000Z,None,2022-10-29T17:12:11.072Z
CVX,convex-finance,cvx,Convex Finance,https://assets.coingecko.com/coins/images/1558...,5.790000,404904833,108,5.787206e+08,7195464.0,5.860000,...,9.553807e+07,1.000000e+08,60.09,-90.40603,2022-01-01T18:04:03.030Z,1.900000,202.95133,2021-07-20T13:17:29.183Z,None,2022-10-29T17:12:08.713Z
1INCH,1inch,1inch,1inch,https://assets.coingecko.com/coins/images/1346...,0.628980,391025599,109,9.438744e+08,30906323.0,0.630424,...,1.500000e+09,1.500000e+09,8.65,-92.73420,2021-10-27T08:24:54.808Z,0.519281,21.05384,2022-06-18T20:10:15.819Z,None,2022-10-29T17:11:59.272Z


In [845]:
chosen_tokens_symbols = list(chosen_tokens.index)
prices_data = prices_data[chosen_tokens_symbols]
marketcaps = marketcaps[chosen_tokens_symbols]

In [846]:
without_nan_index = (marketcaps.isnull().sum(axis=1) == 0) & (prices_data.isnull().sum(axis=1) == 0)
marketcaps = marketcaps[without_nan_index]
prices_data = prices_data[without_nan_index]

## Weighting 

**Weighting requirements**
* The maximum weight any one token can have is 30%.
* All excess weight is proportionally redistributed to all uncapped tokens. After this has been completed, if another token now exceeds the 30% threshold the excess will be redistributed to the remaining uncapped tokens. This process will occur iteratively until there are no tokens that exceed the maximum weight.
* Any asset with a weight below 0.5% will be removed from the index.


### Weight mcaps

In [847]:
weights = marketcaps.div(marketcaps.sum(axis=1), axis=0)
weights = weights.sort_values(weights.last_valid_index(), axis=1, ascending=False)

In [848]:
fig = px.line(weights,
              labels={'value': 'weight, %', 'variable': ''})
fig.update_traces(
    hovertemplate="%{y}"
)
fig.update_yaxes(
    tickformat=".2%",
)
fig.update_xaxes(
    showspikes=True,
    spikethickness=2,
    spikedash="dot",
    spikecolor="#999999",
    spikemode="across",
)
fig.update_layout(
    ## showlegend=False,
    hovermode="x",
    hoverdistance=100,  ## Distance to show hover label of data point
    spikedistance=1000,  ## Distance to show spike
    template='plotly_white',
    title='Weights without max constraint'
)

### Adjust weights

This process adjusts weights to adhere to the weight constraints described in our methodology. In addition the `remove_tiny_weights` function checks for any assets with a weight below the minimum threshold of $0.5\%$ and proportionally increases all remaining weights (which are $> 0.5\%$). Note that after this procedure readjusting weights to respect maximum may be needed.

In [849]:
def adjust_weights(weights, max_weight):
    w = weights.copy()
    
    while (w > max_weight).any(axis=None):
        w[w > max_weight] = max_weight
        c = 1 - w.sum(axis=1)
        w_less = w[w < max_weight]
        w[(w < max_weight) & (w > 0)] += w_less.div(w_less.sum(axis=1), axis=0).mul(c, axis=0)
    
    return w

In [850]:
def remove_tiny_weights(weights, min_weight):
    w = weights.copy()
    
    w[w < min_weight] = 0
    w = w.div(w.sum(axis=1), axis=0)
    
    return w

In [851]:
adjusted_weights = adjust_weights(weights, max_weight=0.3) 
adjusted_weights = remove_tiny_weights(adjusted_weights, min_weight=0.005)
adjusted_weights = adjust_weights(adjusted_weights, max_weight=0.3)

adjusted_weights = adjusted_weights.sort_values(adjusted_weights.last_valid_index(),ascending=False, axis = 1)
# adjusted_weights

In [852]:
fig = px.line(adjusted_weights,
              labels={'value': 'weight, %', 'variable': ''})
fig.update_traces(
    hovertemplate="%{y}"
)
fig.update_yaxes(
    tickformat=".2%",
)
fig.update_xaxes(
    showspikes=True,
    spikethickness=2,
    spikedash="dot",
    spikecolor="#999999",
    spikemode="across",
)
fig.update_layout(
    hovermode="x",
    hoverdistance=100,  ## Distance to show hover label of data point
    spikedistance=1000,  ## Distance to show spike
    template='plotly_white',
    title='Weights with max constraint'
)

### Convert weights to the [1, 255] scale

On the contract side weights are integer numbers from the interval $[1, 255]$ with total sum $255$, so it's needed to convert retrieved weights to this format.

Note that one can't just round weights to integers after scaling, since it doesn't guarantee that their sum will be $255$. To fix that firstly floor function is applied to the weights and then $1$ is being added to the $k$ weights with largest fractional parts.

In [853]:
def convert_weights(weights):
    w_scaled = weights * 255
    w_res = np.floor(w_scaled).astype(int)    
    remainders = w_scaled - w_res    
    k = round(remainders.sum())
    w_below_max = w_res[w_res<76]
    for i in range(k):
        w_below_max[i] +=1
    for i in w_below_max.index:
        w_res[i] = w_below_max[i]
    return w_res

In [854]:
last_weights = adjusted_weights.iloc[-1].dropna()

In [855]:
converted_last_weights = convert_weights(last_weights)

## Summary table

In [856]:
pdi = pd.DataFrame()
pdi.index = chosen_tokens.index
pdi['name'] = chosen_tokens['name']
pdi['market_cap'] = marketcaps.iloc[-1].astype(int)
pdi['price'] = prices_data.iloc[-1]
pdi['weight'] = last_weights
pdi['weight_converted'] = converted_last_weights
pdi['address'] = [top_defi_tokens[symb]['address'] for symb in pdi.index]

pdi = pdi[pdi['weight'] > 0]
pdi = pdi.sort_values("market_cap",ascending=False)
pdi

,name,market_cap,price,weight,weight_converted,address
symbol,,,,,,
UNI,Uniswap,5173563934,6.871136,0.300000,76,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984
LDO,Lido DAO,1205484467,1.681011,0.124904,32,0x5a98fcbea516cf06857215779fd812ca3bef1b32
AAVE,Aave,1189839460,84.127165,0.123283,32,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9
MKR,Maker,811064453,899.707245,0.084037,22,0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2
SNX,Synthetix Network,582652360,2.421560,0.060370,16,0xc011a73ee8576fb46f5e1c5751ca3b9fe0af2a6f
CRV,Curve DAO,530134779,0.944847,0.054929,15,0xd533a949740bb3306d119cc777fa900ba034cd52
RPL,Rocket Pool,468679326,24.527066,0.048561,13,0xd33526068d116ce69f19a9ee46f0bd304f21a51f
CVX,Convex Finance,397451353,5.688693,0.041181,10,0x4e3fbd56cd56c3e72c1403e103b45db9da5b9d2b
1INCH,1inch,381537164,0.614442,0.039532,10,0x111111111117dc0aa78b770fa6a738034120c302


In [857]:
## pdi[['address', 'weight_converted']].to_csv("reweight_info.csv", index=False)
print(pdi[['address', 'weight_converted']].to_csv(None, index=False))

address,weight_converted
0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,76
0x5a98fcbea516cf06857215779fd812ca3bef1b32,32
0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,32
0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2,22
0xc011a73ee8576fb46f5e1c5751ca3b9fe0af2a6f,16
0xd533a949740bb3306d119cc777fa900ba034cd52,15
0xd33526068d116ce69f19a9ee46f0bd304f21a51f,13
0x4e3fbd56cd56c3e72c1403e103b45db9da5b9d2b,10
0x111111111117dc0aa78b770fa6a738034120c302,10
0xc00e94cb662c3520282e6f5717214004a7f26888,9
0x6b3595068778dd592e39a122f4f5a5cf09c90fe2,8
0x0bc529c00c6401aef6d220be8c6ea1667f6ad93e,6
0xba100000625a3754423978a60c9317c58a424e3d,6

